In [1]:
# | default_exp swinv2_3d_with_sdpa

# Imports

In [2]:
# | export

import numpy as np
import torch
import torch.nn.functional as F

from einops import rearrange
from torch import nn
from vision_architectures.swinv2_3d import (
    populate_and_validate_config,
    get_coords_grid,
    SwinV23DMHSA as SwinV23DMHSAWithoutSDPA,
    SwinV23DLayerMLP,
    SwinV23DLayer as SwinV23DLayerWithoutSDPA,
    SwinV23DBlock as SwinV23DBlockWithoutSDPA,
    SwinV23DPatchMerging,
    SwinV23DStage as SwinV23DStageWithoutSDPA,
    SwinV23DEncoder as SwinV23DEncoderWithoutSDPA,
    SwinV23DPatchEmbeddings,
    get_3d_position_embeddings,
    embed_spacings_in_position_embeddings,
    SwinV23DEmbeddings,
    SwinV23DModel as SwinV23DModelWithoutSDPA,
    SwinV23DReconstructionDecoder,
    SwinV23DMIM as SwinV23DMIMWithoutSDPA,
    SwinV23DSimMIM as SwinV23DSimMIMWithoutSDPA,
    SwinV23DVAEMIM as SwinV23DVAEMIMWithoutSDPA,
)

# Modify MHSA

In [3]:
# | export


class SwinV23DMHSA(SwinV23DMHSAWithoutSDPA):
    def __init__(
        self,
        dim,
        num_heads,
        window_size,
        use_relative_position_bias,
        attn_drop_prob=0.0,
        proj_drop_prob=0.0,
    ):
        super().__init__(dim, num_heads, window_size, use_relative_position_bias, attn_drop_prob, proj_drop_prob)

        # Remove attention dropout layer as that is handled automatically, but store the dropout for later
        del self.attn_drop
        self.attn_drop_prob = attn_drop_prob

    def forward(self, hidden_states: torch.Tensor):
        # hidden_states: (windowed_b, window_size_z window_size_y window_size_x, dim)
        _, num_patches_z, num_patches_y, num_patches_x, _ = hidden_states.shape

        query, key, value = rearrange(
            self.W_qkv(hidden_states),
            "b nz ny nx (n num_heads d) -> n b num_heads (nz ny nx) d",
            n=3,
            num_heads=self.num_heads,
        )
        # num_patches = window_size_z * window_size_y * window_size_x
        # Each is (windowed_b, num_heads, num_patches, per_head_dim)

        logit_scale = torch.clamp(self.logit_scale, max=np.log(1.0 / 0.01)).exp()

        query_normalized = F.normalize(query, dim=-1)
        key_normalized = F.normalize(key, dim=-1)

        query_normalized_and_scaled = query_normalized * logit_scale  # Scale the query beforehand

        relative_position_bias = None
        if self.use_relative_position_bias:
            relative_position_bias = self.calculate_relative_position_bias()

        context = F.scaled_dot_product_attention(
            query_normalized_and_scaled,
            key_normalized,
            value,
            attn_mask=relative_position_bias,  # Use this as a way to introduce relative position bias
            dropout_p=self.attn_drop_prob,
            is_causal=False,
            scale=1.0,  # Already scaled the vectors
        )
        # (windowed_b, num_heads, num_patches, per_head_dim)

        context = rearrange(
            context,
            "b num_heads (num_patches_z num_patches_y num_patches_x) d -> "
            "b num_patches_z num_patches_y num_patches_x (num_heads d)",
            num_patches_z=num_patches_z,
            num_patches_y=num_patches_y,
            num_patches_x=num_patches_x,
        )
        # (windowed_b, window_size_z window_size_y window_size_x, dim)

        context = self.proj(context)
        context = self.proj_drop(context)
        # (windowed_b, window_size_z window_size_y window_size_x, dim)

        return context

In [4]:
test = SwinV23DMHSA(54, 6, (4, 4, 4), True)
display(test)
display(test(torch.randn(2, 4, 4, 4, 54)).shape)


SwinV23DMHSA(
  (W_qkv): Linear(in_features=54, out_features=162, bias=True)
  (proj): Linear(in_features=54, out_features=54, bias=True)
  (proj_drop): Dropout(p=0.0, inplace=False)
  (cpb_mlp): Sequential(
    (0): Linear(in_features=3, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=6, bias=False)
  )
)

torch.Size([2, 4, 4, 4, 54])

In [5]:
for use_relative_position_bias in [True, False]:
    m1 = SwinV23DMHSA(54, 6, (4, 4, 4), use_relative_position_bias)
    m2 = SwinV23DMHSAWithoutSDPA(54, 6, (4, 4, 4), use_relative_position_bias)

    m1.load_state_dict(m2.state_dict())
    m1.eval(), m2.eval()

    example_input = torch.randn(2, 4, 4, 4, 54)
    o1 = m1(example_input)
    o2 = m2(example_input)

    assert torch.allclose(o1, o2, atol=1e-6), (o1 - o2).abs().max()

# Modify other classes accordingly

In [6]:
# | export


class SwinV23DLayer(SwinV23DLayerWithoutSDPA):
    def __init__(
        self,
        dim,
        num_heads,
        intermediate_ratio,
        layer_norm_eps,
        window_size,
        use_relative_position_bias,
        attn_drop_prob=0.0,
        proj_drop_prob=0.0,
        mlp_drop_prob=0.0,
    ):
        super().__init__(
            dim,
            num_heads,
            intermediate_ratio,
            layer_norm_eps,
            window_size,
            use_relative_position_bias,
            attn_drop_prob,
            proj_drop_prob,
            mlp_drop_prob,
        )

        self.mhsa = SwinV23DMHSA(
            dim, num_heads, window_size, use_relative_position_bias, attn_drop_prob, proj_drop_prob
        )

In [7]:
# | export


class SwinV23DBlock(SwinV23DBlockWithoutSDPA):
    def __init__(self, stage_config):
        super().__init__(stage_config)

        self.stage_config = stage_config
        self.w_layer = SwinV23DLayer(
            stage_config["_out_dim"],
            stage_config["num_heads"],
            stage_config["intermediate_ratio"],
            stage_config["layer_norm_eps"],
            stage_config["window_size"],
            stage_config["use_relative_position_bias"],
            stage_config.get("attn_drop_prob", 0.0),
            stage_config.get("proj_drop_prob", 0.0),
            stage_config.get("mlp_drop_prob", 0.0),
        )
        self.sw_layer = SwinV23DLayer(
            stage_config["_out_dim"],
            stage_config["num_heads"],
            stage_config["intermediate_ratio"],
            stage_config["layer_norm_eps"],
            stage_config["window_size"],
            stage_config["use_relative_position_bias"],
            stage_config.get("attn_drop_prob", 0.0),
            stage_config.get("proj_drop_prob", 0.0),
            stage_config.get("mlp_drop_prob", 0.0),
        )

In [8]:
# | export


class SwinV23DStage(SwinV23DStageWithoutSDPA):
    def __init__(self, stage_config):
        super().__init__(stage_config)

        self.blocks = nn.ModuleList(
            [SwinV23DBlock(stage_config) for _ in range(stage_config["depth"])],
        )

In [9]:
# | export


class SwinV23DEncoder(SwinV23DEncoderWithoutSDPA):
    def __init__(self, config):
        super().__init__(config)

        self.stages = nn.ModuleList([SwinV23DStage(stage_config) for stage_config in config["stages"]])

In [10]:
# | export


class SwinV23DModel(SwinV23DModelWithoutSDPA):
    def __init__(self, config):
        super().__init__(config)

        self.encoder = SwinV23DEncoder(config)

In [11]:
# | export


class SwinV23DMIM(SwinV23DMIMWithoutSDPA):
    def __init__(self, swin_config, decoder_config, mim_config):
        super().__init__(swin_config, decoder_config, mim_config)

        self.swin = SwinV23DModel(swin_config)

In [12]:
# | export


class SwinV23DSimMIM(SwinV23DSimMIMWithoutSDPA, SwinV23DMIM):
    def __init__(self, swin_config, decoder_config, mim_config):
        super().__init__(swin_config, decoder_config, mim_config)  # This calls all inits in order written above

In [13]:
# | export


class SwinV23DVAEMIM(SwinV23DVAEMIMWithoutSDPA, SwinV23DMIM):
    def __init__(self, swin_config, decoder_config, mim_config):
        super().__init__(swin_config, decoder_config, mim_config)  # This calls all inits in order written above

# Some more tests

### Overfitting tests

In [14]:
from tqdm.auto import tqdm

sample_spacings = torch.tensor([[1, 0.1, 0.1], [2, 0.2, 0.2], [3, 0.3, 0.3], [4, 0.4, 0.4], [5, 0.5, 0.5]])
sample_batch = torch.rand(3, 1, 16, 128, 128)
sample_config = {
    "swin": populate_and_validate_config(
        {
            "patch_size": (1, 4, 4),
            "dim": 12,
            "in_channels": 1,
            "use_absolute_position_embeddings": True,
            "learnable_absolute_position_embeddings": False,
            "embed_spacing_info": False,
            "image_size": (16, 128, 128),
            "drop_prob": 0.2,
            "stages": [
                {
                    "patch_merging": None,
                    "depth": 1,
                    "num_heads": 4,
                    "intermediate_ratio": 4,
                    "layer_norm_eps": 1e-6,
                    "window_size": (4, 4, 4),
                    "use_relative_position_bias": True,
                    "attn_drop_prob": 0.2,
                    "proj_drop_prob": 0.2,
                    "mlp_drop_prob": 0.2,
                },
                {
                    "patch_merging": {
                        "merge_window_size": (2, 2, 2),
                        "out_dim_ratio": 4,
                    },
                    "depth": 3,
                    "num_heads": 4,
                    "intermediate_ratio": 4,
                    "layer_norm_eps": 1e-6,
                    "window_size": (4, 4, 4),
                    "use_relative_position_bias": True,
                },
                {
                    "patch_merging": {
                        "merge_window_size": (2, 2, 2),
                        "out_dim_ratio": 4,
                    },
                    "depth": 1,
                    "num_heads": 4,
                    "intermediate_ratio": 4,
                    "layer_norm_eps": 1e-6,
                    "window_size": (4, 4, 4),
                    "use_relative_position_bias": True,
                },
            ],
        }
    ),
    "mim": {
        "mask_ratio": 0.7,
        "mask_grid_size": (8, 8, 8),
    },
}
sample_config["decoder"] = {
    "dim": sample_config["swin"]["stages"][-1]["_out_dim"],
    "patch_size": sample_config["swin"]["stages"][-1]["_out_patch_size"],
    "in_channels": sample_config["swin"]["in_channels"],
}

model = SwinV23DSimMIM(sample_config['swin'], sample_config['decoder'], sample_config['mim'])

sum(x.numel() for x in model.swin.parameters()), sum(x.numel() for x in model.decoder.parameters())

(1183892, 197632)

In [15]:
from neuro_utils.describe import describe_model

describe_model(model)

Total Parameters: 1,381,536
+---------------------------------------------------------------+------------+
|                             Module                            | Parameters |
+---------------------------------------------------------------+------------+
|                           mask_token                          |     12     |
|    swin.embeddings.patch_embeddings.patch_embeddings.weight   |    192     |
|     swin.embeddings.patch_embeddings.patch_embeddings.bias    |     12     |
|               swin.embeddings.layer_norm.weight               |     12     |
|                swin.embeddings.layer_norm.bias                |     12     |
|    swin.encoder.stages.0.blocks.0.w_layer.mhsa.logit_scale    |     4      |
|    swin.encoder.stages.0.blocks.0.w_layer.mhsa.W_qkv.weight   |    432     |
|     swin.encoder.stages.0.blocks.0.w_layer.mhsa.W_qkv.bias    |     36     |
|    swin.encoder.stages.0.blocks.0.w_layer.mhsa.proj.weight    |    144     |
|     swin.encoder.stage

In [16]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [17]:
sample_batch = sample_batch.cuda()
sample_spacings = sample_spacings.cuda()
model = model.cuda()

In [18]:
for i in tqdm(range(200)):
    optimizer.zero_grad()
    output = model(sample_batch, sample_spacings)
    print(f"Loss: {output[1]:f}\tLR: {scheduler.get_last_lr()[0]:f}")
    output[1].backward()
    optimizer.step()
    scheduler.step()

  0%|          | 0/200 [00:00<?, ?it/s]

Loss: 3.356076	LR: 0.500000
Loss: 3.334477	LR: 0.500000
Loss: 5.702736	LR: 0.500000
Loss: 4.145500	LR: 0.500000
Loss: 3.306134	LR: 0.500000
Loss: 2.851023	LR: 0.450000
Loss: 3.064271	LR: 0.450000
Loss: 1.868436	LR: 0.450000
Loss: 2.064336	LR: 0.450000
Loss: 1.785206	LR: 0.450000
Loss: 1.773572	LR: 0.405000
Loss: 1.483809	LR: 0.405000
Loss: 1.629214	LR: 0.405000
Loss: 1.603664	LR: 0.405000
Loss: 1.426439	LR: 0.405000
Loss: 1.708982	LR: 0.364500
Loss: 1.414643	LR: 0.364500
Loss: 1.447917	LR: 0.364500
Loss: 1.408905	LR: 0.364500
Loss: 1.305879	LR: 0.364500
Loss: 1.424265	LR: 0.328050
Loss: 1.184872	LR: 0.328050
Loss: 1.148759	LR: 0.328050
Loss: 1.217010	LR: 0.328050
Loss: 1.154983	LR: 0.328050
Loss: 1.349384	LR: 0.295245
Loss: 1.119373	LR: 0.295245
Loss: 1.185996	LR: 0.295245
Loss: 1.079443	LR: 0.295245
Loss: 1.110415	LR: 0.295245
Loss: 1.102653	LR: 0.265721
Loss: 1.015544	LR: 0.265721
Loss: 0.958608	LR: 0.265721
Loss: 0.970079	LR: 0.265721
Loss: 0.975689	LR: 0.265721
Loss: 1.048073	LR: 0

In [19]:
for name, param in model.named_parameters():
    if param.grad is None:
        print(name)

In [20]:
sample_config['decoder']['beta'] = None
sample_config['decoder']['beta_schedule'] = (100, 0, 2)

model = SwinV23DVAEMIM(sample_config['swin'], sample_config['decoder'], sample_config['mim'])

encoder_params = sum(x.numel() for x in model.swin.parameters())
decoder_params = sum(x.numel() for x in model.decoder.parameters())
sampling_params = sum(x.numel() for x in model.mu_layer.parameters()) + sum(
    x.numel() for x in model.logvar_layer.parameters()
)
encoder_params, decoder_params, sampling_params

(1183892, 197632, 74112)

In [21]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.9)

In [22]:
sample_batch = sample_batch.cuda()
sample_spacings = sample_spacings.cuda()
model = model.cuda()

In [23]:
for i in tqdm(range(200)):
    optimizer.zero_grad()
    output = model(sample_batch, sample_spacings)
    print(f"Loss: {output[1]:f}\tLR: {scheduler.get_last_lr()[0]:f}\tBeta: {output[3][2]:f}")
    # print(output[-1])
    output[1].backward()
    optimizer.step()
    scheduler.step()

  0%|          | 0/200 [00:00<?, ?it/s]

Loss: 1.494753	LR: 0.001000	Beta: 0.000000
Loss: 4.444311	LR: 0.001000	Beta: 0.020000
Loss: 6.283701	LR: 0.001000	Beta: 0.040000
Loss: 16.800419	LR: 0.001000	Beta: 0.060000
Loss: 9.140267	LR: 0.001000	Beta: 0.080000
Loss: 7.460201	LR: 0.000900	Beta: 0.100000
Loss: 8.268412	LR: 0.000900	Beta: 0.120000
Loss: 9.869583	LR: 0.000900	Beta: 0.140000
Loss: 12.585202	LR: 0.000900	Beta: 0.160000
Loss: 15.908322	LR: 0.000900	Beta: 0.180000
Loss: 13.942308	LR: 0.000810	Beta: 0.200000
Loss: 6.442122	LR: 0.000810	Beta: 0.220000
Loss: 6.089732	LR: 0.000810	Beta: 0.240000
Loss: 6.108383	LR: 0.000810	Beta: 0.260000
Loss: 8.575150	LR: 0.000810	Beta: 0.280000
Loss: 10.641992	LR: 0.000729	Beta: 0.300000
Loss: 9.281338	LR: 0.000729	Beta: 0.320000
Loss: 7.760039	LR: 0.000729	Beta: 0.340000
Loss: 7.441273	LR: 0.000729	Beta: 0.360000
Loss: 9.329135	LR: 0.000729	Beta: 0.380000
Loss: 10.819846	LR: 0.000656	Beta: 0.400000
Loss: 9.125205	LR: 0.000656	Beta: 0.420000
Loss: 5.875267	LR: 0.000656	Beta: 0.440000
Loss:

In [24]:
for name, param in model.named_parameters():
    if param.grad is None:
        print(name)

# nbdev

In [26]:
!nbdev_export

# Rough work